In [9]:
import pandas as pd
import numpy as np

df = pd.DataFrame({'x':[0.0, 1.0, np.nan, -1.0], 
                   'y':[0.0, np.nan, np.nan, "foo"],
                   'z':[0,   1,      2,       3]})
df

,x,y,z
0,0.0,0,0
1,1.0,NaN,1
2,NaN,NaN,2
3,-1.0,foo,3


The behavior of subsetting of `df` is undefined. Sometimes `df[]` returns a copy, other times it returns a view. This can lead to unexpected behavior because modifying a view will change the original `df`, but modifying a copy will not. 

The only time subsetting is guaranteed to return a view and allow modifying df is when there is a single subsetting operation on the lefb hand side of an assignment, `df[] = `.

This is why the following works:

In [10]:
df.loc[1,'x'] = 2
df

,x,y,z
0,0.0,0,0
1,2.0,NaN,1
2,NaN,NaN,2
3,-1.0,foo,3


but chained subsetting gives a warning 

In [11]:
df['x'][1] = 3
df

<ipython-input-11-68904c52768e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['x'][1] = 3


,x,y,z
0,0.0,0,0
1,3.0,NaN,1
2,NaN,NaN,2
3,-1.0,foo,3


It happened to modify `df` too, which is likely what is desired. However, there is no guarantee that this will always happen. In other contexts or in different versions of python and/or pandas, `df['x'][1]` might return a copy and modifying the copy will have no effect on the original `df`.

For this reason, subsetting should never be combined with `inplace` operations. 

In [12]:
# Subsetting with an array of logicals creates a copy
x = df['x'].copy()
mask = df['z']>1
x[mask].fillna(0,inplace=True)
print(x)

# Subsetting with an array of ints creates a copy
print(np.where(mask)[0])
x[np.where(mask)[0]].fillna(0,inplace=True)
print(x)

# Subsetting with a range returns a view, but this behavior is not guaranteed
x[2:4].fillna(0,inplace=True)
print(x) # finally x modified

0    0.0
1    3.0
2    NaN
3   -1.0
Name: x, dtype: float64
[2 3]
0    0.0
1    3.0
2    NaN
3   -1.0
Name: x, dtype: float64
0    0.0
1    3.0
2    0.0
3   -1.0
Name: x, dtype: float64


The face that last subsetting with a range allowed modifying the original `x` in place should not be relied upon. It may change in future versions of python/pandas/numpy. 

Arguably, subsetting with inplace operations should also through a `SettingWithCopyWarning`.